In [1]:
import os
import numpy as np
import pytorch_lightning as pl
import torch.nn as nn
import torch
from data import LabeledDataset
from torch.utils.data import DataLoader
from CFS_SG import CFS_SG

In [2]:
import matplotlib
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import svd
from scipy.linalg import pinv
from itertools import product

data = np.genfromtxt('Data_Cortex_Nuclear.csv',delimiter=',',
                     skip_header=1,usecols=range(1,78),filling_values=0)
classes = np.genfromtxt('Data_Cortex_Nuclear.csv',delimiter=',',
                        skip_header=1,usecols=range(78,81),dtype=None)

data_pd = pd.read_csv("Data_Cortex_Nuclear.csv")

target_idx_A = np.where((classes[:,-1]==b'S/C') & (classes[:,-2]==b'Saline') & (classes[:,-3]==b'Control'))[0]
target_idx_B = np.where((classes[:,-1]==b'S/C') & (classes[:,-2]==b'Saline') & (classes[:,-3]==b'Ts65Dn'))[0]

labels = len(target_idx_A)*[0] + len(target_idx_B)*[1]
target_idx = np.concatenate((target_idx_A,target_idx_B))                                                                          

target = data[target_idx]

background_idx = np.where((classes[:,-1]==b'C/S') & (classes[:,-2]==b'Saline') & (classes[:,-3]==b'Control'))
background = data[background_idx]

foreground_gene_names = data_pd.columns.values[1:-4]


/tmp/ipykernel_2379909/1128385069.py:12: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  classes = np.genfromtxt('Data_Cortex_Nuclear.csv',delimiter=',',


In [3]:
labels_train = np.concatenate([np.zeros(background.shape[0]), np.ones(target.shape[0])])
data_train = np.concatenate([background, target])

data_train = torch.from_numpy(data_train).float()  
labels_train = torch.from_numpy(labels_train).float()  

# Create the dataset as before
dataset = LabeledDataset(data_train.numpy(), labels_train.numpy())


In [4]:
input_size = target.shape[1]
output_size = background.shape[1]
batch_size = 128

In [5]:
model = CFS_SG(
    input_size=input_size,
    output_size=output_size,
    hidden=[512, 512], # Number of units in each hidden layer
    k_prime=20, # Background dimension size
    lam=0.15, # Tuned to select about 10 features
    lr=1e-3,
    loss_fn=nn.MSELoss()
)


In [6]:
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

trainer = pl.Trainer(max_epochs=10, accelerator='gpu', devices=1)
trainer.fit(model, loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/nas/longleaf/home/eyzhang/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA A100-PCIE-40GB MIG 2g.10gb') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.htm

Training: |          | 0/? [00:00<?, ?it/s]

/nas/longleaf/home/eyzhang/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:212: You called `self.log('num_selected_gates', ...)` in your `training_step` but the value needs to be floating to be reduced. Converting it to torch.float32. You can silence this warning by converting the value to floating point yourself. If you don't intend to reduce the value (for instance when logging the global step or epoch) then you can use `self.logger.log_metrics({'num_selected_gates': ...})` instead.
`Trainer.fit` stopped: `max_epochs=10` reached.


In [7]:
indices = model.get_inds(10) 

In [8]:
foreground_gene_names[indices]

array(['DYRK1A_N', 'MEK_N', 'pERK_N', 'TRKA_N', 'NR2A_N', 'pPKCG_N',
       'pNR2B_N', 'pMTOR_N', 'BAD_N', 'pP70S6_N'], dtype=object)